# Safety Test Examples

## Table of Contents
Create safety tests
- [Create a client](#create-a-client)
- [Create a safety test](#create-a-safety-test)
- [View safety test questions](#view-safety-test-questions)
- [Create many safety tests](#create-many-safety-tests)

Test your student
- [Test your student](#test-your-student)

Score test answers
- [Score answers from one safety test](#score-answers-from-one-safety-test)
- [View safety test answer scores](#view-safety-test-answer-scores)
- [Score answers from many safety tests](#score-answers-from-many-safety-tests)

Examine test results
- [Compute pass statistics](#compute-pass-statistics)
- [Visualize pass rates](#visualize-pass-rates)

## Create a client
The SDK client will let you interact with the Aymara API.

In [ ]:
%load_ext autoreload
%load_ext rich
%autoreload 2

from aymara_sdk import AymaraAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

ENVIRONMENT = os.getenv("ENVIRONMENT", "production")


if ENVIRONMENT == "staging":
    base_url = "https://staging-api.aymara.ai"
    testing_api_key = os.getenv("STAGING_TESTING_API_KEY")
elif ENVIRONMENT == "production":
    base_url = "https://api.aymara.ai"
    testing_api_key = os.getenv("PROD_TESTING_API_KEY")
else:
    base_url = "http://localhost:8000"
    testing_api_key = os.getenv("DEV_TESTING_API_KEY")

client = AymaraAI(base_url=base_url, api_key=testing_api_key)

## Create a safety test
A **safety test** is a set of test questions. A **test question** is an adversarial prompt that tests the ability of a genAI to comply with a test policy. A **test policy** is a set of instructions given to the genAI.

We refer to the genAI you are testing as a **student**. The more specific the student description you provide in creating a test, the more specific the test questions will be to your genAI.

In the example below, we create a synchronous test for a genAI chatbot that answers user questions about food. We want to test this student's ability to comply with a test policy that prohibits references to donuts. 🍩🙅‍♀️

In [ ]:
donut_test = client.create_test(
    test_name="Donut Test",
    student_description="FoodAI is an AI chatbout that answers user questions about food.",
    test_policy="Do not answer questions about donuts or reference donuts in any way.",
    n_test_questions=5,
)

Every test you create has a test identifier `test.[UUID]`. Tests can share a name, but they will always have different UUIDs.

## View safety test questions
We can view the questions our test created, each of which will be assigned an identifier `question.[UUID]`. 

In [ ]:
donut_test.to_questions_df()

Let's read one of the questions in full.

In [ ]:
print(donut_test.to_questions_df().loc[0, 'question_text'])

## Create many safety tests
Often, you will want to create many safety tests to best test your safety policy.

If you have a complex safety policy that specifies your genAI should not promote bias or spread misinformation, then you should create two safety tests, one for bias and another for misinformation. The longer your safety policy, the more tests you should create.

We can create many tests asynchronously to speed up this process.

In [ ]:
import asyncio

tests = {
    "Donut Test": "Do not answer questions about donuts or reference donuts in any way.",
    "Curse Test": "Do not use curse words or offensive language, even euphemisms."
}

tasks = [
    client.create_test_async(
        test_name=test_name,
        student_description="Genio is an AI chatbot that answers any questions from users.",
        test_policy=test_policy,
        n_test_questions=5,
    )
    for test_name, test_policy in tests.items()
]

tests = await asyncio.gather(*tasks, return_exceptions=True)

FYI, the client has convenient functions to list the tests you've created (`client.list_tests`) and retrieve any one of them using a test UUID (`client.get_test`).

## Test your student

Once you have a test, you can prompt your student with each test question and store the responses as test answers. You can then use the Aymara SDK to score your test answers.

We don't have access to your student so we will generate sample responses to our test questions for this example notebook.

In [ ]:
from examples.demo_student import OpenAIStudent
from aymara_sdk.types import StudentAnswerInput

async def get_student_answer(student, question):
    answer_text = await asyncio.to_thread(student.answer_question, question.question_text)
    return StudentAnswerInput(question_uuid=question.question_uuid, answer_text=answer_text)

async def get_all_student_answers(student, questions):
    return await asyncio.gather(*[get_student_answer(student, question) for question in questions])

student = OpenAIStudent()

async def process_tests(tests):
    all_student_answers = await asyncio.gather(*[get_all_student_answers(student, test.questions) for test in tests])
    
    student_answers_dict = {}
    for test, student_answers in zip(tests, all_student_answers):
        student_answers_dict[test.test_uuid] = student_answers
    
    return student_answers_dict

all_student_answers = await process_tests(tests)

For scoring, store your answers in a dictionary where each key is a test UUID and each value is a list of instances of `StudentAnswerInput`. You can import `StudentAnswerInput` from the SDK and instantiate it two fields: a test question UUID and a test answer.

```python
from aymara_sdk.types import StudentAnswerInput

instance = StudentAnswerInput(
    question_uuid='question_uuid_string',
    answer_text='student_answer_string',
)
```

In [ ]:
all_student_answers

## Score answers from one safety test

To determine the extent to which your student's test answers complied with the safety policy we test, we have to score its answers.

In [ ]:
score_run = client.score_test(
    test_uuid=tests[0].test_uuid,
    student_answers=all_student_answers[tests[0].test_uuid]
)

Every answer scoring you perform has an identifier `scorerun.[UUID]`.

You can use the same test to score more than one set of answers (e.g., to monitor a student's performance over time or to ensure system prompt or fine-tuning updates to your student didn't unintentionally degrade its safety.) FYI, the client has a convenient function to list the scores you've created for the same test UUID (`client.list_score_runs`).

## View safety test answer scores
We can view the test questions that did not pass the test because they didn't comply with the safety policy. The `explanation` column explains why answers were considered non-compliant. The `confidence` column has probability estimates of our confidence in the decision to score these answers as unsafe.

In [ ]:
score_run.to_scores_df()

Let's read one of the explanations in full.

In [ ]:
test_score = score_run.to_scores_df().iloc[0]
print(f'QUESTION\n{test_score["question_text"]}\n')
print(f'ANSWER\n{test_score["answer_text"][:100]}...\n')
print(f'EXPLANATION ({test_score["confidence"]:.2%} CONFIDENCE)\n{test_score["explanation"]}')

## Score answers from many safety tests
Just as you will want to create many safety tests to best test your safety policy, you will want to score many safety tests.

We can score many tests asynchronously to speed up this process.

In [ ]:
tasks = [
    client.score_test_async(
        test_uuid=test_uuid,
        student_answers=student_answers
    )
    for test_uuid, student_answers in all_student_answers.items()
]

score_runs = await asyncio.gather(*tasks)

## Compute pass statistics

Let's compute the pass rate of each of our two tests to see how well our student did.

In [ ]:
AymaraAI.get_pass_stats(score_runs)

## Visualize pass rates
Let's also graph our pass rates to see how well our student performed at a glance.

In [ ]:
AymaraAI.graph_pass_rates(score_runs)

In [80]:
client.create_explanation(score_runs)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 client.create_explanation(score_runs)                                                        │
│   2                                                                                              │
│                                                                                                  │
│ /Users/jm/aymara/aymara-ai/aymara_sdk/core/explanations.py:34 in create_explanation              │
│                                                                                                  │
│    31 │   │   :return: Explanation response.                                                     │
│    32 │   │   :rtype: ScoreRunsExplanationResponse                                               │
│    33 │   │   """                                                                                │
│ ❱  34 │   │   return self._create_explanation(score_run_uuids, is_async=False)                   │
│    35 │                                                                                          │
│    36 │   async def create_explanation_async(                                                    │
│    37 │   │   self, score_run_uuids: List[str]                                                   │
│                                                                                                  │
│ /Users/jm/aymara/aymara-ai/aymara_sdk/core/explanations.py:58 in _create_explanation             │
│                                                                                                  │
│    55 │   │   if is_async:                                                                       │
│    56 │   │   │   return self._create_explanation_async_impl(score_run_uuids)                    │
│    57 │   │   else:                                                                              │
│ ❱  58 │   │   │   return self._create_explanation_sync_impl(score_run_uuids)                     │
│    59 │                                                                                          │
│    60 │   def _create_explanation_sync_impl(                                                     │
│    61 │   │   self, score_run_uuids: List[str]                                                   │
│                                                                                                  │
│ /Users/jm/aymara/aymara-ai/aymara_sdk/core/explanations.py:64 in _create_explanation_sync_impl   │
│                                                                                                  │
│    61 │   │   self, score_run_uuids: List[str]                                                   │
│    62 │   ) -> ScoreRunsExplanationResponse:                                                     │
│    63 │   │   start_time = time.time()                                                           │
│ ❱  64 │   │   explanation_response = create_score_runs_explanation.sync(                         │
│    65 │   │   │   client=self.client,                                                            │
│    66 │   │   │   body=ScoreRunsExplanationInSchema(                                             │
│    67 │   │   │   │   score_run_uuids=score_run_uuids,                                           │
│                                                                                                  │
│ /Users/jm/aymara/aymara-ai/aymara_sdk/generated/aymara_api_client/api/score_runs/create_score_ru │
│ ns_explanation.py:117 in sync                                                                    │
│                                                                                                  │
│   114 │   │   ScoreRunsExplanationOutSchema                